In [1]:
import torch
import torchvision as tv
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn

In [2]:
train_data=tv.datasets.MNIST(".",download=True,train=True,transform=ToTensor())
test_data=tv.datasets.MNIST(".",download=True,train=False,transform=ToTensor())

In [3]:
print(len(train_data),len(test_data))

60000 10000


In [4]:
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(test_data,batch_size=32,shuffle=False)

In [5]:
itr=iter(train_loader)
b,l=next(itr)
print(type(b),b.view(32,-1).size())

<class 'torch.Tensor'> torch.Size([32, 784])


In [10]:
input_size=28*28
output_size=10
hidden_size1=128
hidden_size2=64
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.layer1=nn.Linear(input_size,hidden_size1)
    self.layer2=nn.Linear(hidden_size1,hidden_size2)
    self.layer3=nn.Linear(hidden_size2,output_size)
    self.relu=nn.ReLU()
  def forward(self,x):
    x=x.view(x.size()[0],-1)
    x=self.layer1(x)
    x=self.relu(x)
    x=self.layer2(x)
    x=self.relu(x)
    x=self.layer3(x)

    return x


In [11]:
model=Net()
print(model)

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)


In [12]:
def accuracy(outputs,labels):
    _,pred=torch.max(outputs,dim=1)
    acc=torch.sum(pred==labels).item()
    return torch.tensor(acc/len(labels))
def validate(model,batch):
    imgs,labels=batch
    imgs=imgs.cuda()
    labels=labels.cuda()
    outputs=model(imgs)
    acc=accuracy(outputs,labels)
    return acc 
@torch.no_grad() 
def evaluate(model,loader):
    model.eval()
    acc=[validate(model,batch) for batch in loader]
    acc=torch.stack(acc).mean()
    return acc 

In [13]:
model=model.cuda()
import torch.optim as optim
epochs=10
optimizer=optim.SGD(model.parameters(),lr=0.01)
loss_fn=nn.CrossEntropyLoss()
running_loss=0.0
for epoch in range(epochs):
  model.train()
  for i,data in enumerate(train_loader):
    optimizer.zero_grad()
    img,label=data
    img,label=img.cuda(),label.cuda()
  
    output=model(img)
    loss=loss_fn(output,label)
  
    loss.backward()
    optimizer.step()
    running_loss=0.99*running_loss+0.01*loss.item()
  print("loss {:.4f}".format(running_loss))
  acc=evaluate(model,test_loader)
  print(acc)

loss 0.4979
tensor(0.8717)
loss 0.3716
tensor(0.9007)
loss 0.3006
tensor(0.9206)
loss 0.2587
tensor(0.9283)
loss 0.2439
tensor(0.9366)
loss 0.2100
tensor(0.9381)
loss 0.1973
tensor(0.9433)
loss 0.1715
tensor(0.9500)
loss 0.1557
tensor(0.9540)
loss 0.1488
tensor(0.9537)
